Import

In [3]:
import json
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
from haversine import haversine

Load data

In [4]:
# Load data from the CSV file into a new DataFrame
data = pd.read_csv('../model/gps_data2.csv')

Extract nodes

In [5]:
unique_taxi_ids = data['Taxi ID'].unique()
unique_taxi_ids_df = pd.DataFrame({'Taxi ID': unique_taxi_ids})
unique_taxi_ids_df.to_csv('../files/nodes/tdrive.csv', index=False)

Calculate distance function

In [6]:
def calculate_distance(coord1, coord2):
	return haversine(coord1, coord2)

Save close coordinates

In [7]:
meeting_points = defaultdict(list)
meeting_distance_threshold = 0.1
meeting_info_list = []

for index, row in tqdm(data.iterrows(), desc="main data"):
	taxi_id = row["Taxi ID"]
	location = (row["Latitude"], row["Longitude"])

	if meeting_points[location]:
		for other_taxi_id in meeting_points[location]:
			if taxi_id != other_taxi_id:
				other_location = (data.loc[data["Taxi ID"] == other_taxi_id, "Latitude"].iloc[0], data.loc[data["Taxi ID"] == other_taxi_id, "Longitude"].iloc[0])

				distance = calculate_distance(location, other_location)
				if distance <= meeting_distance_threshold:
					meeting_info = {
						"Taxi1": int(other_taxi_id),
						"Taxi2": int(taxi_id),
						"Location": {
							"Latitude": location[0],
							"Longitude": location[1]
						},
						"Distance (meters)": distance,
						"Timestamp1": row["Timestamp"],
						"Timestamp2": data.loc[data["Taxi ID"] == other_taxi_id, "Timestamp"].iloc[0]
					}

					meeting_info_list.append(meeting_info)

	meeting_points[location].extend([taxi_id])

main data: 0it [00:00, ?it/s]

main data: 859082it [04:30, 3171.40it/s] 


KeyboardInterrupt: 

Save into a file

In [8]:
with open('../files/edges/tdrive.json', 'w') as json_file:
    json.dump(meeting_info_list, json_file, indent=4)